<a href="https://colab.research.google.com/github/gulabpatel/Python_Tutorials/blob/master/Colab_Special/Getting_started_with_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


### Provide your credentials to the runtime

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [2]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

# Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [3]:
# Display query output immediately

%%bigquery --project gulab-project
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

,total_rows
0,114420316


In [4]:
# Save output in a variable `df`

%%bigquery --project gulab-project df
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

In [5]:
df

,total_rows
0,114420316


# Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://bigquery.cloud.google.com/table/bigquery-public-data:samples.gsod) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Declare the Cloud project ID which will be used throughout this notebook

In [8]:
project_id = 'gulab-project'

### Sample approximately 2000 random rows

In [9]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 114420316 rows


### Describe the sampled data

In [10]:
df.describe()

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,total_precipitation,snow_depth
count,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,1942.000000,1942.000000,1539.000000,1539.000000,739.000000,739.000000,1818.000000,1818.000000,2000.000000,2000.000000,1969.000000,287.000000,2019.000000,1834.000000,111.000000
mean,504803.291584,90812.016832,1987.000495,6.445050,15.874752,52.226337,13.267327,41.590937,13.116375,1014.751787,11.632878,970.801759,12.736130,11.951705,12.732673,6.719650,13.186500,12.060843,25.063763,43.621793,0.079858,10.357658
std,297004.249472,25735.409615,16.403424,3.415353,8.799492,24.890740,7.940206,23.431491,7.947201,9.531897,7.659257,60.624782,7.983854,8.813293,7.816768,4.909989,7.918843,6.645503,8.876338,24.877250,0.336929,11.054054
min,10015.000000,3104.000000,1933.000000,1.000000,1.000000,-50.400002,4.000000,-57.900002,4.000000,916.400024,4.000000,676.200012,4.000000,0.000000,4.000000,0.000000,4.000000,1.900000,3.900000,-65.000000,0.000000,0.400000
25%,243395.000000,99999.000000,1977.000000,3.000000,8.000000,38.000000,7.000000,29.700001,7.000000,1009.350006,6.000000,958.600006,7.000000,6.400000,6.000000,3.400000,7.000000,7.800000,19.000000,30.800000,0.000000,3.100000
50%,513755.000000,99999.000000,1989.000000,6.000000,16.000000,55.900002,8.000000,43.599998,8.000000,1014.400024,8.000000,995.000000,8.000000,9.600000,8.000000,5.700000,8.000000,11.100000,24.100000,46.400002,0.000000,7.100000
75%,724795.500000,99999.000000,2001.000000,9.000000,24.000000,71.450001,24.000000,58.074999,24.000000,1020.000000,22.000000,1009.500000,24.000000,14.800000,23.000000,8.900000,24.000000,15.500000,29.900000,61.500000,0.010000,15.000000
max,999999.000000,99999.000000,2010.000000,12.000000,31.000000,97.099998,24.000000,81.900002,24.000000,1062.000000,24.000000,1032.599976,24.000000,99.400002,24.000000,57.900002,24.000000,57.900002,69.900002,91.000000,6.300000,74.800003


### View the first 10 rows

In [11]:
df.head(10)

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,max_temperature_explicit,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,724048,99999,1977,4,4,50.099998,5,NaN,NaN,NaN,NaN,NaN,NaN,4.8,5.0,NaN,NaN,NaN,NaN,45.900002,True,None,None,NaN,NaN,False,False,False,False,False,False
1,30170,99999,2006,5,17,48.200001,23,45.900002,23.0,NaN,NaN,NaN,NaN,6.9,23.0,9.4,23.0,19.0,25.1,45.000000,True,None,None,0.08,NaN,True,True,True,True,True,True
2,100180,99999,2008,10,21,54.000000,10,44.400002,10.0,NaN,NaN,NaN,NaN,6.9,10.0,18.6,10.0,22.9,34.0,51.799999,True,None,None,NaN,NaN,False,False,False,False,False,False
3,103800,99999,1954,11,23,23.500000,4,21.000000,4.0,1019.599976,4.0,NaN,NaN,1.9,4.0,12.5,4.0,14.0,NaN,21.000000,False,None,None,0.00,NaN,False,False,False,False,False,False
4,309610,99999,1971,3,1,-11.800000,6,-22.500000,6.0,1041.900024,6.0,NaN,NaN,31.1,6.0,4.4,5.0,9.9,NaN,-22.000000,True,None,None,0.00,NaN,False,False,False,False,False,False
5,298650,99999,1991,7,9,82.300003,6,63.299999,6.0,1001.299988,6.0,NaN,NaN,15.8,6.0,2.3,6.0,3.9,NaN,69.300003,True,None,None,0.00,NaN,False,False,False,False,False,False
6,591580,99999,2007,1,29,55.799999,7,39.900002,7.0,1024.400024,7.0,1014.000000,7.0,6.6,7.0,2.8,7.0,3.9,NaN,48.000000,True,None,None,0.00,NaN,False,False,False,False,False,False
7,401650,99999,1958,8,10,79.599998,8,64.900002,8.0,1007.099976,8.0,NaN,NaN,7.5,8.0,4.9,8.0,13.0,NaN,70.000000,True,None,None,0.00,NaN,False,False,False,False,False,False
8,67050,99999,1992,8,3,75.300003,14,59.599998,14.0,1013.299988,14.0,964.700012,14.0,NaN,NaN,6.0,14.0,9.9,NaN,63.700001,True,None,None,0.00,NaN,False,False,False,False,False,False
9,716279,99999,1985,4,16,56.200001,16,43.099998,16.0,1005.299988,16.0,NaN,NaN,10.4,16.0,9.9,16.0,20.0,31.1,44.599998,True,None,None,0.00,NaN,True,True,True,True,True,True


In [12]:
# 10 highest total_precipitation samples
df.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

,station_number,year,month,day,total_precipitation
357,969950,2001,3,19,6.30
1786,265240,1967,2,23,5.91
1671,222130,1988,11,11,4.72
469,152380,1995,12,21,4.02
1611,478170,1990,3,28,3.35
1753,945890,1992,3,29,2.52
1132,32120,1995,10,3,2.09
1326,483150,2003,8,8,2.08
367,723490,1983,10,4,2.06
96,676730,1996,3,15,2.01


# Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [13]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id=project_id, dialect='standard')

df.head()

,name,count
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028


# Syntax highlighting
`google.colab.syntax` can be used to add syntax highlighting to any Python string literals which are used in a query later.

In [14]:
from google.colab import syntax
query = syntax.sql('''
SELECT
  COUNT(*) as total_rows
FROM
  `bigquery-public-data.samples.gsod`
''')

pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,total_rows
0,114420316
